# 📊 Microsoft Fabric Monitoring Notebook

This notebook automates the collection of runtime metadata for key Fabric artifacts:

- **Semantic Models**: via Power BI REST API
- **Data Pipelines**: via Microsoft Fabric REST API

The results are consolidated and stored in a **Delta Lake table**, enabling performance and error trend monitoring across workspaces.

## 🔍 Functionality

- Authenticates using `notebookutils.credentials.getToken('pbi')`
- Iterates through all available Fabric workspaces
- Collects run metadata for:
  - **Semantic Models** (last refresh via Power BI)
  - **Data Pipelines** (last run via Fabric Job Scheduler)
- Outputs to: `dbo.item_runs` (Delta Lake table)

Links used:
  - **Semantic Models**: [Get Refresh History](https://learn.microsoft.com/en-us/rest/api/power-bi/datasets/get-refresh-history)
  - **Data Pipelines**: [List Item Job Instances](https://learn.microsoft.com/en-us/rest/api/fabric/core/job-scheduler/list-item-job-instances)
  - **Workspace List**: [List Workspaces](https://learn.microsoft.com/en-us/rest/api/fabric/core/workspaces/list-workspaces)
  - **Workspace Items**: [List Items in a Workspace](https://learn.microsoft.com/en-us/rest/api/fabric/core/items/list-items)

## 🧾 Output Schema

The Delta table (`dbo.item_runs`) has the following schema:

| Column                 | Type      | Description                           |
|------------------------|-----------|---------------------------------------|
| `workspace_name`       | string    | Fabric workspace name                 |
| `workspace_id`         | string    | Fabric workspace GUID                 |
| `item_type`            | string    | Type: `SemanticModel` or `DataPipeline` |
| `item_name`            | string    | Display name of the item              |
| `item_id`              | string    | GUID of the item                      |
| `last_run_date_time`   | timestamp | Time of most recent run/refresh       |
| `last_run_duration_ms` | long      | Duration in milliseconds              |
| `last_run_status`      | string    | Status: `Completed`, `Failed`, etc.   |
| `is_error`             | int       | 1 if run failed, 0 otherwise          |
| `last_run_request_id`  | string    | Unique request ID from the API        |


> Use this notebook for scheduled or on-demand diagnostics in your Fabric platform.


In [ ]:
item_runs_table_name = 'dbo.item_runs'
data_pipelines_table_name = 'dbo.data_pipeline_status'
monitored_item_types = ['SemanticModel', 'DataPipeline']

access_token = notebookutils.credentials.getToken('pbi')

In [ ]:

from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, IntegerType
from datetime import datetime

schema = StructType([
    StructField("workspace_name", StringType(), True),
    StructField("workspace_id", StringType(), True),
    StructField("item_type", StringType(), True),
    StructField("item_name", StringType(), True),
    StructField("item_id", StringType(), True),
    StructField("last_run_date_time", TimestampType(), True),
    StructField("last_run_duration_ms", LongType(), True),
    StructField("last_run_status", StringType(), True),
    StructField("is_error", IntegerType(), True),
    StructField("last_run_request_id", StringType(), True)
])

items_history = []

In [ ]:
import requests

def call_fabric_api(url):
    # Set headers with authorization
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    # Make the GET request
    response = requests.get(url, headers=headers)

    try:
        data = response.json()
    except ValueError:
        print(f"Error: Unable to parse JSON. Status: {response.status_code}, Response: {response.text}")
        print(f"Url: {url}")
        return

    # Check response code
    if response.status_code != 200:        
        print(f"Error or unexpected response format. Status: {response.status_code}, Response: {data}")
        return
        
    # Check if 'value' exists and is a list
    if isinstance(data.get("value"), list):
        return data["value"]
    else:
        return data




In [ ]:
def process_item_run(workspace_name, workspace_id, item_name, item_id, item_type, url_template, time_fields, id_field):
    url = url_template.format(workspace_id=workspace_id, item_id=item_id)
    run_history = call_fabric_api(url)

    if not run_history:
        print(f"⚠️ No run history found for {item_name} ({item_type}) in {workspace_name}")
        return {
            "workspace_name": workspace_name,
            "workspace_id": workspace_id,
            "item_type": item_type,
            "item_name": item_name,
            "item_id": item_id,
            "last_run_date_time": None,
            "last_run_duration_ms": None,
            "last_run_status": None,
            "is_error": None,
            "last_run_request_id": None
        }

    run = run_history[0]
    start_key, end_key = time_fields

    start_time = datetime.fromisoformat(run[start_key])
    end_time = datetime.fromisoformat(run[end_key]) if run.get(end_key) else None
    duration_ms = int((end_time - start_time).total_seconds() * 1000) if end_time else None
    status = run.get("status", "In Progress")

    return {
        "workspace_name": workspace_name,
        "workspace_id": workspace_id,
        "item_type": item_type,
        "item_name": item_name,
        "item_id": item_id,
        "last_run_date_time": end_time,
        "last_run_duration_ms": duration_ms,
        "last_run_status": status,
        "is_error": 1 if status == "Failed" else 0,
        "last_run_request_id": run.get(id_field)
    }


In [ ]:
# List workspaces
workspaces = call_fabric_api("https://api.fabric.microsoft.com/v1/workspaces")


In [ ]:


# Loop through workspaces, for each item get runs and save to an array
for workspace in workspaces:

    workspace_name = workspace['displayName']
    workspace_id = workspace['id']

    print(f"🔍 Processing workspace: {workspace_name}({workspace_id})")

    # List data items in workspace
    items = call_fabric_api(f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items")

    if not items:
        print(f"No items found for workspace {workspace_name} ({workspace_id}).")
        continue


    # Build a set of all item names in the workspace excluding SemanticModels and Reports
    non_semantic_model_names = {
        i['displayName'] for i in items if i['type'] not in {'SemanticModel', 'Report'}
    }


    # Loop through items and get last run info
    for item in items:

        item_name = item['displayName']
        item_id = item['id']
        item_type = item['type']   

        # Exclude default semantic models - name must match
        if item_type == "SemanticModel" and item_name in non_semantic_model_names:
            print(f"Skip default SemanticModel '{item_name}' in {workspace_name} (auto-created)")
            continue
             
        # Filter only for selected item types
        if item_type not in monitored_item_types:
            print(f"Skip:{item_name} in {workspace_name} due to item type{item_type}")
            continue

        print(f"🔍 Processing item: {item_name}({item_id}) -Type: {item_type} in workspace: {workspace_name}")

        match item_type:
            case "SemanticModel":
                item_data = process_item_run(
                    workspace_name,
                    workspace_id,
                    item_name,
                    item_id,
                    item_type,
                    url_template="https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/datasets/{item_id}/refreshes?$top=1",
                    time_fields=("startTime", "endTime"),
                    id_field="requestId"
                )
            case "DataPipeline":
                item_data = process_item_run(
                    workspace_name,
                    workspace_id,
                    item_name,
                    item_id,
                    item_type,
                    url_template="https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{item_id}/jobs/instances",
                    time_fields=("startTimeUtc", "endTimeUtc"),
                    id_field="id"
                )
            case _:
                log(f"Skipping {item_name}: unsupported type {item_type}")
                continue


        if not item_data:
            continue


        items_history.append(item_data)
    



In [ ]:
# write to Delta

spark.createDataFrame(items_history, schema) \
    .write.mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(item_runs_table_name)
